In [277]:
import re
import json
import csv
import numpy as np
from Index import Index 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
import pandas as pd
from matplotlib import pyplot as plt
import math
from rank_bm25 import BM25Okapi

# Data Extraction

In [278]:
def read(f):
    content = ''
    for line in f:
        content += line
    return content 

In [279]:
with open('cisi\\CISI.ALL', 'r') as f:
    content = read(f)
titles = [x.groups()[0] for x in re.finditer(r"\.T[^\.\w]*?\n((.|\n)*?)\n\.A", content)]
abstracts = [x.groups()[0] for x in re.finditer(r"\.W.*?\n((.|\n)*?)\n\.X", content)]

In [280]:
with open('cisi\\CISI.QRY', 'r') as f:
    content = read(f)
queries = [x.groups()[0] for x in re.finditer(r"\.W[^\.\w]*?\n((.|\n)*?)\n\.", content)]

In [281]:
len(titles), len(abstracts), len(queries)

(1460, 1460, 112)

In [282]:
docs = [t + ' ' + a for t, a in zip(titles, abstracts)]

# Building Dataset

In [283]:
index = Index(docs) 
index.process() 

Tokenizing...


100%|██████████| 1460/1460 [00:04<00:00, 342.77it/s]


Removing empty words...


100%|██████████| 1460/1460 [00:00<00:00, 2402.70it/s]


Getting frequencies...


...

1460it [00:05, 250.34it/s]


Getting weights...


100%|██████████| 1460/1460 [00:00<00:00, 2140.20it/s]


Combining...


1460it [00:05, 276.00it/s]


## Index and Inverted

In [284]:
index.get_index()

In [285]:
index.get_inverted()

In [286]:
with open('index.json', 'w', encoding='utf-8') as f:
    json.dump(index.index, f, ensure_ascii=False, indent=4)

In [287]:
with open('inverted.json', 'w', encoding='utf-8') as f:
    json.dump(index.inverted, f, ensure_ascii=False, indent=4)

## CSV File

In [288]:
data = []
for k in list(index.index.keys()):
    for v in index.index[k]:
        data.append([k] + v)

In [289]:
with open("dataset.csv", "wt", newline='') as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(['Document', 'Token', 'Frequency', 'Weight']) 
    writer.writerows(data)

## Queries

In [290]:
def filter(token):
    t = PorterStemmer().stem(token)
    return WordNetLemmatizer().lemmatize(t)

In [291]:
def tokenize(docs, regex='(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%?|\w+(?:\-\w+)*'):
    regex = nltk.RegexpTokenizer(regex) 
    tokens_lists = [regex.tokenize(txt) for txt in docs]
    tokens_lists = [[filter(t) for t in tokens_list] for tokens_list in tokens_lists] 
    empty_words = nltk.corpus.stopwords.words('english')
    tokens_lists = [[token.lower() for token in tokens if token not in empty_words] for tokens in tokens_lists]
    return tokens_lists

In [292]:
queries = tokenize(queries)
queries = [list(np.unique(q)) for q in queries]

In [293]:
queries = {i: q for i, q in enumerate(queries)} 

In [294]:
with open('queries.json', 'w', encoding='utf-8') as f:
    json.dump(queries, f, ensure_ascii=False, indent=4)

## Ground truth

In [295]:
f = open('cisi\\CISI.REL', 'r')
truth = [x.split()[:2] for x in f.readlines()]
with open("ground_truth.csv", "wt", newline='') as fp:
    writer = csv.writer(fp, delimiter=",")
    writer.writerow(['Query', 'Relevent document']) 
    writer.writerows(truth)

# Predictions

In [316]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from tqdm import tqdm

import numpy as np

class Index():
    def __init__(self, docs, preprocessed=False):
        if preprocessed:
            index, inverted, queries, ground_truth = docs
            self.index = index
            self.inverted = inverted 
            self.queries = queries
            self.ground_truth = ground_truth
        else:
            self.docs = docs
        self.wnl = WordNetLemmatizer()
        self.port = PorterStemmer()        

    def filter(self, token):
        t = self.port.stem(token)
        return self.wnl.lemmatize(t)

    def tokenize(self, regex='(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%?|\w+(?:\-\w+)*'):
        regex = nltk.RegexpTokenizer(regex) 
        self.tokens_lists = [regex.tokenize(txt) for txt in self.docs]
        self.tokens_lists = [[self.filter(t) for t in tokens_list] for tokens_list in tqdm(self.tokens_lists)] 
        empty_words = nltk.corpus.stopwords.words('english')
        print('Removing empty words...')
        self.tokens_lists = [[token.lower() for token in tokens if token not in empty_words] for tokens in tqdm(self.tokens_lists)]
    
    def get_freq(self):
        self.frequencies = []
        for tokens in tqdm(self.tokens_lists):
            dict = {}
            for token in tokens:
                dict[token] = (dict[token] if token in dict.keys() else 0) + 1
            self.frequencies.append(dict) 

    def get_weights(self):
        max = [np.max(list(d.values())) for d in self.frequencies]
        self.weights = []
        for i in tqdm(range(len(max))):
            d = {}
            for k, v in self.frequencies[i].items():
                d[k] = round((v/max[i]) * np.log10(len(max)/len(self.get_docs(k, preprocessed=True))+1), 2)
            self.weights.append(d)
        
    def combine(self, origin):
        out = {}
        sets = set()
        for o in origin:
            sets = sets | set(o)
        frequencies = [{k: origin[i].get(k) if origin[i].get(k) else  0 for k in sets} for i in range(len(origin))]
        for freq, d in tqdm(zip(frequencies, range(len(frequencies)))):
            for k, v in freq.items():
                out[(k, d)] = v
        return out

    def process(self):
        print('Tokenizing...')
        self.tokenize()
        print('Getting frequencies...')
        self.get_freq()
        print('Combining...')
        self.all_frequencies = self.combine(self.frequencies) 
        self.get_inverted_f()
        print('Getting weights...')
        self.get_weights()
        print('Combining...')
        self.all_weights = self.combine(self.weights) 
        self.get_index()
        self.get_inverted()
        
    def get_index(self):
        index = {}
        for doc, (w, f) in enumerate(zip(self.weights, self.frequencies)):
            d = []
            for token in list(w.keys()):
                d.append([token, f[token], w[token]])
            index[doc] = d
        self.index = index

    def get_inverted_f(self):
        inverted = {}
        for doc, f in enumerate(self.frequencies):
            for token in list(f.keys()):
                if token in inverted.keys():
                    inverted[token].append([doc, f[token]]) 
                else:
                    inverted[token] = [[doc, f[token]]]
        self.inverted = inverted

    def get_inverted(self):
        inverted = {}
        for doc, (w, f) in enumerate(zip(self.weights, self.frequencies)):
            for token in list(w.keys()):
                if token in inverted.keys():
                    inverted[token].append([doc, f[token], w[token]]) 
                else:
                    inverted[token] = [[doc, f[token], w[token]]]
        self.inverted = inverted
        
    def get_docs_per_token(self, token):
        f = {d: v for (t, d), v in self.all_frequencies.items() if token == t} 
        f = [i for i in list(f.values()) if i != 0]
        return f 
    
    def get_docs(self, token, preprocessed=False):
        if not preprocessed:
            token = self.filter(token.lower())
        return self.inverted[token]

    def get_docs_query(self, query):
        tokens = [token for token in query.split()]
        all = {}
        details = {}
        for token in tokens:
            docs = self.get_docs(token)
            for d in docs:
                if d[0] not in all.keys():
                    all[d[0]] = [[d[1]], [d[2]]]
                else:
                    all[d[0]] = [[all[d[0]][0][0] + d[1]], [round(all[d[0]][1][0] + d[2], 2)]]
            details[token] = {d[0]: [d[1], d[2]] for d in docs}
        return details, all

    def scalar_prod(self, n_doc, query):
        result = 0
        for token in query:
            try:
                result += [l[2] for l in self.get_docs(token, preprocessed=True) if l[0] == n_doc][0]
            except:
                pass
        return result

    def cosine_measure(self, doc, query):
        w = [self.get_docs(token, preprocessed=True)[0][2] for token in doc]
        result = np.sqrt(len(query)) * np.sqrt(np.dot(w, w))
        return self.scalar_prod(doc, query) / result 

    def jaccard_measure(self, doc, query):
        w = [self.get_docs(token, preprocessed=True)[0][2] for token in doc]
        result = len(query) + np.dot(w, w) - self.scalar_prod(doc, query)
        return self.scalar_prod(doc, query) / result

    def vector_search(self, max_docs=50, metric='scalar'):
        queries = np.unique(self.ground_truth['Query'])
        relevent_docs = [list(self.ground_truth[self.ground_truth['Query'] == q]['Relevent document']) for q in queries]
        predicted = {} 
        if metric == 'scalar':
            metric = self.scalar_prod
        elif metric == 'cosine':
            metric = self.cosine_measure
        elif metric == 'jaccard':
            metric = self.jaccard_measure
        for q in tqdm(queries):
            pred = []
            query = self.queries[str(q-1)] 
            for doc, tokens in self.index.items():
                try:
                    pred.append([q, doc, metric(doc, query)]) 
                except:
                    pred.append([q, doc, metric([t[0] for t in tokens], query)])
            pred = sorted(pred, key=lambda x: x[2], reverse=True)
            predicted[q] = [p[1] for p in pred[:max_docs]] 
        return predicted.values(), relevent_docs

    def PR(self, pred, relevent):
        precisions = []
        recalls = []
        len_TP = []
        len_pred = []
        for p, r in zip(pred, relevent):
            TP = len(set(p) & set(r))
            precisions.append(TP/len(p))
            recalls.append(TP/len(r)) 
            len_TP.append(TP)
            len_pred.append(len(p))
        return np.mean(len_TP), np.mean(len_pred), np.mean(precisions), np.mean(recalls)

    def accuracy(self, pred, relevent):
        acc = []
        for p, r in zip(pred, relevent):
            TP = len(set(p) & set(r))
            acc.append(TP/len(r))
        return np.mean(acc)

    def BM25_per_doc(self, n_doc, doc, query, avgdl, N, b=0.75, k1=1.2):
        score = 0
        for token in query:
            try:
                temp = self.get_docs(token, preprocessed=True)
            except:
                temp = None
            try:
                tf = [l[2] for l in temp if l[0] == n_doc][0]
            except:
                tf = 0
            dl = len(doc)
            n = 0
            try:
                n = len(temp)
            except:
                pass
            idf = math.log((N - n + 0.5) / (n + 0.5))
            w = idf * (tf * (k1 + 1) / (tf + k1 * (1 - b + b * dl / avgdl)))
            score += w
        return score

    def BM25(self):
        pred = []
        queries = np.unique(self.ground_truth['Query'])
        N = len(self.index)
        avgdl = sum([len(t) for _, t in self.index.items()]) / N
        for q in tqdm(queries):
            query = self.queries[str(q-1)] 
            scores = []
            for doc, tokens in self.index.items():
                scores.append(self.BM25_per_doc(doc, tokens, query, avgdl, N))
            pred.append(sorted(zip(range(N), scores), key=lambda x: x[1], reverse=True))
        return pred

    def new_bm25(self):
        bm25 = BM25Okapi([[i[0] for i in j] for j in list(self.index.values())])
        queries = np.unique(self.ground_truth['Query'])
        pred = []
        for q in tqdm(queries):
            query = self.queries[str(q-1)] 
            pred.append(sorted(enumerate(bm25.get_scores(query)), key=lambda x: x[1], reverse=True)) 
        return pred

    '''def bool_search(self, q):
        keywords = ['and', 'or', 'not']
    
        q = q.replace('-', 'not ')
        print() 
        print('q', q) 

        parsed = [self.filter(token.lower()) for token in q.split()]
        parsed = [i for i in parsed if i not in keywords]
        print('parsed', parsed) 
        parsed = [index.filter(i.lower()) for i in parsed]

        _, freq, _ = self.get_freq_tokens(' '.join(parsed))
        freq = {k: {d: 1 if f else 0 for d, f in v.items()} for k, v in freq.items()}

        print('freq', freq)    
        q = q.lower()
        r = [q, q, q, q] 

        for k, v in freq.items():
            for d, f in v.items():
                r[d] = r[d].replace(k, str(f))

        print('r', r)'''

    def bool_search(self, q):
        keywords = ['and', 'or', 'not']
        
        q = q.replace('-', 'not ')
        q = ' '.join([self.filter(token.lower()) for token in q.split()])
    
        print()
        print('q', q)

        parsed = [self.filter(token.lower()) for token in q]
        parsed = [i for i in parsed if i not in keywords]
        print('parsed', parsed)
        parsed = [index.filter(i.lower()) for i in parsed]

        _, freq, _ = self.get_freq_tokens(' '.join(parsed))
        freq = {k: {d: 1 if f else 0 for d, f in v.items()} for k, v in freq.items()}

        print('freq', freq)    

        r = [q, q, q, q] 

        for k, v in freq.items():
            for d, f in v.items():
                r[d] = r[d].replace(k, str(f))
        print('r', r)

In [317]:
with open('queries.json') as f:
    queries = json.load(f) 
ground_truth = pd.read_csv('ground_truth.csv', sep=',')
index = Index((index.index, index.inverted, queries, ground_truth), preprocessed=True)

## Vector

In [298]:
predicted, relevent_docs = index.vector_search(max_docs=20, metric='scalar') 
index.PR(list(predicted), relevent_docs) 

100%|██████████| 76/76 [00:46<00:00,  1.64it/s]


(1.0789473684210527, 20.0, 0.053947368421052626, 0.030589365932939688)

In [299]:
predicted, relevent_docs = index.vector_search(max_docs=200, metric='cosine')
index.PR(list(predicted), relevent_docs) 

100%|██████████| 76/76 [01:00<00:00,  1.26it/s]


(8.144736842105264, 200.0, 0.04072368421052632, 0.2133032768890171)

In [300]:
predicted, relevent_docs = index.vector_search(max_docs=200, metric='jaccard') 
index.PR(list(predicted), relevent_docs) 

100%|██████████| 76/76 [01:53<00:00,  1.49s/it]


(8.144736842105264, 200.0, 0.04072368421052632, 0.2133032768890171)

## BM25

In [319]:
# ChatGPT's
p = index.BM25() 
index.PR([[i[0] for i in j[:50]] for j in p], relevent_docs) 

100%|██████████| 76/76 [01:02<00:00,  1.22it/s]


(2.276315789473684, 50.0, 0.04552631578947369, 0.06604444158123768)

In [318]:
# Dude on Kaggle's
p = index.new_bm25()
index.PR([[i[0] for i in j[:50]] for j in p], relevent_docs) 

100%|██████████| 76/76 [00:00<00:00, 92.61it/s] 


(2.6842105263157894, 50.0, 0.05368421052631579, 0.07072597750776016)